In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

edge_driver_path = r"C:\Users\adity\Downloads\msedgedriver.exe"

service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

# Load the webpage
url = "https://www.skyquestt.com/report/ent-devices-market"

driver.get(url)

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "nav-tabs"))
)
page_source1 = driver.page_source
toc_tab = driver.find_element(By.CSS_SELECTOR, "a[href='#tab_default_3']")
toc_tab.click()

WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "tab_default_3"))
)

toc_element = driver.find_element(By.ID, "tab_default_3")

WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CLASS_NAME, "special-toc-class"))
)

page_source = driver.page_source

soup = BeautifulSoup(page_source, "html.parser")

toc_section = soup.find("div", {"class": "special-toc-class"})
if toc_section:

    print(toc_section)

driver.quit()

<div class="special-toc-class"><ul>
<li><strong>Introduction</strong></li>
<li>
<ul>
<li>Objectives of the Study</li>
<li>Definitions</li>
<li>Market Scope</li>
</ul>
</li>
<li><strong>Research Methodology</strong></li>
<li>
<ul>
<li>Information Procurement</li>
<li>Secondary &amp; Primary Data Sources</li>
<li>Market Size Estimation</li>
<li>Market Assumptions &amp; Limitations</li>
</ul>
</li>
<li><strong>Executive Summary</strong></li>
<li>
<ul>
<li>Market Overview Outlook</li>
<li>Supply-Demand Trend Analysis</li>
<li>Segmental Opportunity Analysis</li>
</ul>
</li>
<li><strong>Market Dynamics &amp; Outlook</strong></li>
<li>
<ul>
<li>Market Dynamics
<ul>
<li>Drivers</li>
<li>Opportunities</li>
<li>Restraints</li>
<li>Challenges</li>
</ul>
</li>
<li>Porter’s Analysis
<ul>
<li>Competitive Rivalry</li>
<li>Threat of Substitute Products</li>
<li>Bargaining Power of Buyers</li>
<li>Threat of New Entrants</li>
<li>Bargaining Power of Suppliers</li>
</ul>
</li>
</ul>
</li>
<li><strong>Key

In [2]:
def extract_bullet_points(ul_element, level=0):
    bullet_symbols = ["•", "o", ""]
    bullet_list = []

    for li in ul_element.find_all("li", recursive=False):
        bullet_symbol = bullet_symbols[min(level, len(bullet_symbols) - 1)]

        bullet_text = []

        for content in li.contents:
            if isinstance(content, str):
                text_content = content.strip()
                if text_content:
                    bullet_text.append(text_content)
            elif content.name == "strong":
                bullet_text.append(content.get_text(strip=True))
            elif content.name == "ul":
                if bullet_text:
                    bullet_list.append(
                        f"{bullet_symbol} {' '.join(bullet_text).strip()}"
                    )
                bullet_list.extend(extract_bullet_points(content, level + 1))
                bullet_text = []

        if bullet_text:
            bullet_list.append(f"{bullet_symbol} {' '.join(bullet_text).strip()}")

    return bullet_list


if toc_section:
    ul_element = toc_section.find("ul")
    if ul_element:
        bullet_points = extract_bullet_points(ul_element)

        # Print the bullet points
        for bullet in bullet_points:
            print(bullet)
    else:
        print("No UL found inside the ToC section.")
else:
    print("ToC not found!")

• Introduction
o Objectives of the Study
o Definitions
o Market Scope
• Research Methodology
o Information Procurement
o Secondary & Primary Data Sources
o Market Size Estimation
o Market Assumptions & Limitations
• Executive Summary
o Market Overview Outlook
o Supply-Demand Trend Analysis
o Segmental Opportunity Analysis
• Market Dynamics & Outlook
o Market Dynamics
 Drivers
 Opportunities
 Restraints
 Challenges
o Porter’s Analysis
 Competitive Rivalry
 Threat of Substitute Products
 Bargaining Power of Buyers
 Threat of New Entrants
 Bargaining Power of Suppliers
• Key Market Insights
o Key Success Factors
o Value Chain Analysis
o Top Investment Pockets
o Market Attractive Index
o Ecosystem Mapping
o Patent Analysis
o Macroeconomic Indicators
o Trade Analysis
o Supply Chain Analytics
o Pricing Analysis
o Technology Analysis
o Case Study Analysis
o Customer Buying Criteria
o Regulatory Analysis
• Global Structural Adhesive Market by Substrate
o Market Overview
o Wood
o Plast

In [15]:
bullet_points_str = "\n".join(bullet_points)
lines = bullet_points_str.strip().split("\n")
toc_content = "\n".join(lines)

In [16]:
soup = BeautifulSoup(page_source1, "html.parser")
print(soup)

<html lang="en"><head>
<!-- Google Tag Manager -->
<!--<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WDC9HJ9G');</script>-->
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="1TlU84698yi5CZKlKX6b5vJe7XnDbYZWGa1XT6Oo" name="csrf-token"/>
<title>Automotive Wiring Harness Market Growth, Size, Share &amp; Trends | 2031</title>
<link href="https://www.skyquestt.com/assets/backend/images/favicon.ico" rel="icon"/>
<meta content="Automotive Wiring Harness Market Growth, Size, Share &amp; Trends | 2031" name="title"/>
<meta content="Automotive Wiring Harness Market Growth, Size, Share &amp; Trends | 2031" property="og:title"/>
<meta

In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")


def AI(text, instruct):
    openai_client = OpenAI(api_key=api_key)

    prompt = f"{text}"
    instruction = f"""{instruct}"""

    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
        ],
    )
    assistant_response = completion.choices[0].message.content
    return assistant_response

In [7]:
description = soup.find("div", class_="report-details-description")

first_para = description.find("p").text.strip()

print(first_para)
market_name = first_para.split("Market", 1)[0].strip()
print(market_name)
all_paragraphs = description.find_all("p")

remaining_paragraphs = [para.text for para in all_paragraphs[1:]]

remaining_text = "\n".join(remaining_paragraphs)
remaining_text_instruction = "Rephrase and put in one paragraph i need 250 words"
second_para = AI(remaining_text, remaining_text_instruction).strip()
print(second_para)
third_para = f"""
Top-down and bottom-up approaches were used to estimate and validate the size of the {market_name} market and to estimate the size of various other dependent submarkets. The research methodology used to estimate the market size includes the following details: The key players in the market were identified through secondary research, and their market shares in the respective regions were determined through primary and secondary research. This entire procedure includes the study of the annual and financial reports of the top market players and extensive interviews for key insights from industry leaders such as CEOs, VPs, directors, and marketing executives. All percentage shares split, and breakdowns were determined using secondary sources and verified through Primary sources. All possible parameters that affect the markets covered in this research study have been accounted for, viewed in extensive detail, verified through primary research, and analyzed to get the final quantitative and qualitative data.
""".strip()
print(third_para)
forth_para = f"{market_name} Market Segmental Analysis".strip()
print(forth_para)
fifth_para = soup.select_one("#tab_default_1 > div:nth-of-type(3) > p").text.strip()
print(fifth_para)

sixth_para = f"Driver of the {market_name} Market".strip()
print(sixth_para)
ninth_div = soup.select_one("#tab_default_1 > div:nth-of-type(9)")
driver_inst = f"rephrase this market is {market_name} market driver i need 100 words in one paragraph"
if ninth_div:
    drivers = ninth_div.select_one("ul:nth-of-type(1)")
    if drivers:
        seventh_para = AI(drivers.text, driver_inst).strip()
print(seventh_para)
eighth_para = f"Restraints in the {market_name} Market".strip()
print(eighth_para)

Global Dental Imaging Market size was valued at USD 3.09 billion in 2022 and is poised to grow from USD 3.37 billion in 2023 to USD 6.71 billion by 2031, growing at a CAGR of 9.0% in the forecast period (2024-2031).
Global Dental Imaging
Dental imaging is a vital technique that generates high-quality images for the diagnosis of various dental issues, enabling dentists to visually assess problems such as concealed structures, cavities, bone loss, and oral cancers. The dental imaging equipment market is experiencing robust growth due to several factors, including the rising demand for cosmetic dentistry, improved affordability of imaging systems, and the need for accurate and rapid diagnoses. This market is further fueled by an increasing number of dental practices, advancements in technology, higher dental spending, and a growing prevalence of dental disorders. In 2021, the dental imaging equipment accounted for approximately 2% of the US$ 156.4 billion dentistry market, reflecting its 

In [12]:
ninth_div = soup.select_one("#tab_default_1 > div:nth-of-type(9)")
print(ninth_div)
ninth_inst = f"rephrase this market is {market_name} market restraint i need 100 words in one paragraph"
if ninth_div:
    restraints_para = None
    for p in ninth_div.find_all("p"):
        if "restraint" in p.text.lower():
            restraints_para = p
            break

    if restraints_para:
        nextul = restraints_para.find_next_sibling("ul")
        ninth_para = AI(nextul.text, ninth_inst).strip()
        print(ninth_para)

<div>
<p><strong>Dental Imaging Market Drivers</strong></p>
<ul>
<li>Dental caries, commonly known as tooth decay or periodontal disease, is the world's most common chronic disease and a major global public health concern, affecting people of all ages and stages of life. Dental caries is still a serious oral health concern in most developed countries, affecting the majority of children and adults. Indigenous people in multiple countries, including Australia, Canada, the United States, New Zealand, and Brazil, have lower oral health outcomes and self-reported oral health difficulties due to the influence of various social determinants of health and limited timely access to care.</li>
</ul>
<p><strong>Dental Imaging Market Restraint </strong></p>
<ul>
<li><a id="_heading=h.4d34og8"></a>Lack of dental services is one of the most significant impediments to oral health care. Because oral health is not universally incorporated into primary or behavioural healthcare services, a huge proportio

In [ ]:
tenth_para = f"Market Trends of the {market_name} Market".strip()
print(tenth_para)
eleven_div = soup.select_one("#tab_default_1 > div:nth-of-type(11)")
eleven_inst = f"rephrase this market is {market_name} market trend i need 100 words in one paragraph  "
if eleven_div:
    ul_eleven = eleven_div.select_one("ul:nth-of-type(1)")
    if ul_eleven:
        ul_text = ul_eleven.text.strip()
        eleven_para = AI(ul_text, eleven_inst).strip()
        print(eleven_para)
    else:
        print("No <ul> found in the 11th <div>.")
else:
    print("No 11th <div> found in #tab_default_1.")


description_content = "\n\n".join(
    [
        first_para,
        second_para,
        third_para,
        forth_para,
        fifth_para,
        sixth_para,
        seventh_para,
        eighth_para,
        ninth_para,
        tenth_para,
        eleven_para,
    ]
)

In [8]:
head_div = soup.find(
    "div", class_="d-sm-flex flex-sm-row-reverse align-items-center title"
)

title = head_div.find("h1").text.strip()

print(title)

Structural Adhesive Market Size, Share, Growth Analysis, By Resin (Epoxy, Polyurethane), By Technology (Water-based, Solvent-based), By Substrate (Plastic, Metal), By Application (Transportation, Construction), By Region - Industry Forecast 2024-2031


In [18]:
import re

code = soup.find("div", class_="report-segment-data max-width-640")
report_id_tag = code.find("b", string="Report ID:")

if report_id_tag:
    product_code = (
        report_id_tag.next_sibling.strip() if report_id_tag.next_sibling else ""
    )
    product_code = re.sub(r"\W+", "", product_code)
    print(f"Product Code: {product_code}")
else:
    print("Report ID not found.")

report_len_tag = code.find("b", string="Pages:")
if report_len_tag:
    len = report_len_tag.next_sibling.strip() if report_len_tag.next_sibling else ""
    length = re.sub(r"\W+", "", len)
    print(f"length: {length}")

Product Code: SQMIG25A2164
length: 157


In [19]:
sec = soup.find("ol", class_="MuiBreadcrumbs-ol css-nhb8h9")
sect = sec.find_all("li", class_="MuiBreadcrumbs-li")[1]
sector = sect.text.strip()
print(sector)

Consumer Discretionary


In [20]:
ten_div = soup.select_one("#tab_default_1 > div:nth-of-type(10)")

compa = ten_div.find("ul")

companies_list = []

if compa:
    for li in compa.find_all("li"):
        companies_list.append(f"◦ {li.text.strip()}")

cell_companies = "\n".join(companies_list)

In [21]:
seg = soup.find("td", class_="fw-bold", string="Segments covered")
next_td = seg.find_next_sibling()
segments_list = []
next_td_ul = next_td.find("ul")

next_td_li = next_td_ul.find_all("li", recursive=False)

for li in next_td_li:
    main_category = li.contents[0].strip()

    subcategory = li.find("ul")

    if subcategory:
        subcategory_items = [item.strip() for item in subcategory.stripped_strings]
        subcategory_text = ", ".join(subcategory_items)

        formatted_output = f"By {main_category} ({subcategory_text})"
    else:
        formatted_output = f"By {main_category}"

    segments_list.append(formatted_output)
products = ", ".join(segments_list)

print(products)

By Category (General Wires, Heat-Resistant Wires, Shielded Wires, Tubed Wires), By Propulsion Type (ICE, EV), By Application, for ICE Vehicles (Seat Harness, Engine Harness, Chassis Harness, Body & Lighting Harness, HVAC Harness, Dashboard/Cabin Harness, Battery Harness, Sunroof Harness, Door Harness), By Component, for ICE Vehicles (Connectors, Wires, Terminals, Others), By Material Type, for ICE Vehicles (Metallic (Copper, Aluminum, Other Materials), Optical Fiber (Plastic Optical Fiber (POF), Glass Optical Fiber (GOF))), By ICE Vehicles Type and Application (Passenger Car (Engine Harness, Chassis Harness, Body & Lighting Harness, HVAC Harness, Dashboard/Cabin Harness, Battery Harness, Seat Harness, Sunroof Harness, Door Harness), Light Commercial Vehicle (Engine Harness, Chassis Harness, Body & Lighting Harness, HVAC Harness, Dashboard/Cabin Harness, Battery Harness, Door Harness), Bus (Engine Harness, Chassis Harness, Body & Lighting Harness, HVAC Harness, Dashboard/Cabin Harness, 

In [25]:
import re

print(first_para)
value_pattern = re.compile(
    r"USD (\d+\.?\d*)\s*(Billion|Million|Trillion|billion|million|trillion)"
)
year_pattern = re.compile(r"\b(2022|2023|2031)\b")
cagr_pattern = re.compile(r"CAGR of (\d+\.?\d*)%")

currency_values = value_pattern.findall(first_para)

years = year_pattern.findall(first_para)

cagr = cagr_pattern.search(first_para)

data_2022 = None
data_2023 = None
data_2031 = None
currency = None

if currency_values:
    data_2022 = currency_values[0][0]
    data_2023 = currency_values[1][0]
    data_2031 = currency_values[2][0]
    currency = "USD " + currency_values[0][1]

cagr_value = cagr.group(1) + "%"

print(f"{data_2022} ")
print(f"{data_2023} ")
print(f"{data_2031}")
print(f"{cagr_value}")
print(currency)

Global Dental Imaging Market size was valued at USD 3.09 billion in 2022 and is poised to grow from USD 3.37 billion in 2023 to USD 6.71 billion by 2031, growing at a CAGR of 9.0% in the forecast period (2024-2031).
3.09 
3.37 
6.71
9.0%
USD billion


In [14]:
from openpyxl import Workbook
from openpyxl.styles import Alignment

countries_list = [
    "USA",
    "Canada",
    "Germany",
    "Spain",
    "Italy",
    "France",
    "UK",
    "China",
    "India",
    "Japan",
    "South Korea",
    "Brazil",
    "GCC Countries",
    "South Africa",
]

formatted_countries = [f"◦ {country}" for country in countries_list]

cell_countries = "\n".join(formatted_countries)

In [15]:
price_single = "5300"
price_sitelesense = "6200"
price_enterprise = "7100"